In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import KNNImputer

In [2]:
# Cargamos los datos.
train_data = pd.read_csv(r'C:\Users\rodri\Desktop\Titanic 2.0\datos\train.csv')
test_data = pd.read_csv(r'C:\Users\rodri\Desktop\Titanic 2.0\datos\test.csv')

---

<center><h1>Imputación para la columna 'Age'.</h1></center>

Sabemos que la columna 'Age' tiene los siguientes valores nulos:

Datos faltantes en el dataset de entrenamiento: 177/891 (casi el 20%).  
Datos faltantes en el dataset de prueba: 86/418 (aproximadamente el 20%).  

Para resolver este problema, voy a usar KNN Imputer para estimar los valores faltantes de la edad en función de otras variables como 'Pclass', 'Sex', 'SibSp', 'Parch' y 'Fare'.

In [3]:
imputer = KNNImputer(n_neighbors = 5)

In [4]:
# Convertimos la columna 'Sex' a valor numérico asignando 0 a los hombres y 1 a las mujeres.
train_data['Sex'] = train_data['Sex'].map({'male': 0, 'female': 1})
test_data['Sex'] = test_data['Sex'].map({'male': 0, 'female': 1})

In [5]:
# Imputamos 'Age' en el dataset de entrenamiento.
train_data['Age'] = imputer.fit_transform(train_data[['Age', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Fare']])[:, 0]

# Imputamos 'Age' en el dataset de prueba.
test_data['Age'] = imputer.fit_transform(test_data[['Age', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Fare']])[:, 0]

In [6]:
# Revisamos los valores faltantes en 'Age'.
print(f"Valores nulos en 'Age':\nEntrenamiento: {train_data['Age'].isnull().sum()} | Prueba: {test_data['Age'].isnull().sum()}")

Valores nulos en 'Age':
Entrenamiento: 0 | Prueba: 0


---

<center><h1>Imputación para la columna 'Cabin'.</h1></center>

Sabemos que la columna 'Cabin' tiene los siguientes valores nulos:

Datos faltantes en el dataset de entrenamiento: 687/891 (casi el 77%).  
Datos faltantes en el dataset de prueba: 327/418 (aprox. el 78%).

Ya que la cantidad de valores nulos dentro de esta variable, la imputación puede que no sea efectiva. Normalmente, si una columna no es necesaria para nuestros analisis o modelos, se elimina. Pero también podríamos crear una nueva variable binaria donde 1 indique que el pasajero tiene un número de cabina conocido, y 0 que no.

In [7]:
# Creamos la columna 'TieneCabina' como variable binaria.
train_data['TieneCabina'] = train_data['Cabin'].apply(lambda x: 0 if pd.isnull(x) else 1)
test_data['TieneCabina'] = test_data['Cabin'].apply(lambda x: 0 if pd.isnull(x) else 1)

Como se comentó anteriormente, existe la posibilidad de que eliminemos estas columnas si es que no tienen algún impacto significativo en los análisis.

---

<center><h1>Imputación para la columna 'Embarked'.</h1></center>

Sabemos que la columna 'Embarked' tiene los siguientes valores nulos:

Datos faltantes en el dataset de entrenamiento: 2/891

Como los valores nulos en esta columna son solo 2, podemos simplemente imputar los valores faltantes con la **moda** (valor más frecuente).

### Observación.

Para realizar este paso, se intentó usar el siguiente código:

embarked_mas_frecuente = train_data['Embarked'].mode()[0]  
train_data['Embarked'].fillna(embarked_mas_frecuente, inplace = True)

El cual no tuvo problemas en hacer lo que se busca, pero pandas me imprimió un mensaje al correr el código:

**FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.**
**The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.**

**For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.**

Así que vamos a usar una de las dos recomendaciones que nos aparece en el mensaje.

In [8]:
embarked_mas_frecuente = train_data['Embarked'].mode()[0]
train_data['Embarked'] = train_data['Embarked'].fillna(embarked_mas_frecuente)

---

<center><h1>Imputación para la columna 'Fare'.</h1></center>

Sabemos que la columna 'Fare' tiene los siguientes valores nulos:

Datos faltantes en el dataset de prueba: 1/418

Al ser un valor númerico y solo tener 1 dato faltante, podemos imputarlo con la mediana de las tarifas.

In [9]:
fare_mediana = test_data['Fare'].median()
test_data['Fare'] = test_data['Fare'].fillna(fare_mediana)

---

### Verificamos los valores nulos de ambos datasets.

In [10]:
print('Valores nulos en el dataset de entrenamiento:')
train_data.isnull().sum()

Valores nulos en el dataset de entrenamiento:


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
TieneCabina      0
dtype: int64

In [11]:
print('Valores nulos en el dataset de prueba:')
test_data.isnull().sum()

Valores nulos en el dataset de prueba:


PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
TieneCabina      0
dtype: int64

*Nota importante*

Recordemos que la columna 'Cabin' no fue imputada, sino que se creó una columna binaria que indica con 1 si el pasajero tiene un número de cabina conocido, o 0 si no lo tiene.

---

### Guardamos los datasets procesados.

In [12]:
train_data.to_csv(r'\Users\rodri\Desktop\Titanic 2.0\datos\train_sin_nulos.csv', index = False)
test_data.to_csv(r'\Users\rodri\Desktop\Titanic 2.0\datos\test_sin_nulos.csv', index = False)